# 20241005
1. m3s표기되게 변경



In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton, QLabel, QComboBox
from PyQt5.QtWidgets import QHBoxLayout
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar


# 세로선을 계산하는 함수
def calculate_vlines(center, pitch, max_die_x, min_die_x):
    lines = []
    current = center
    for _ in range(max_die_x + 2):
        lines.append(current)
        current += pitch
    current = center
    for _ in range(abs(min_die_x) + 1):
        current -= pitch
        lines.append(current)
    return lines

# 가로선을 계산하는 함수
def calculate_hlines(center, pitch, max_die_y, min_die_y):
    lines = []
    current = center
    for _ in range(max_die_y + 2):
        lines.append(current)
        current += pitch
    current = center
    for _ in range(abs(min_die_y) + 1):
        current -= pitch
        lines.append(current)
    return lines

# 통계 계산 함수 (예: ABS(Mean) + 3sigma)
def calculate_statistics(values):
    mean_val = np.mean(values)
    sigma_val = np.std(values)
    m3s_val = np.abs(mean_val) + 3 * sigma_val
    
    # 계산된 값을 nm 단위로 변환 (1e6 곱하기)
    m3s_nm = m3s_val * 1e3
    

    return m3s_nm




# Overlay를 플롯하는 함수
def plot_overlay(ax, x, y, dx, dy, v_lines, h_lines, wafer_radius=150000, title='Wafer Vector Map', scale_factor=1e-7):
    quiver = ax.quiver(x, y, dx, dy, angles='xy', scale_units='xy', scale=scale_factor, color='blue', label='Overlay Vectors',
              width=0.0015, headwidth=3, headlength=3)

    ax.axvline(0, color='red', linewidth=1.0, label='Central X')
    ax.axhline(0, color='red', linewidth=1.0, label='Central Y')

    for vline in v_lines:
        ax.axvline(vline, color='black', linestyle='--', linewidth=0.8)
    for hline in h_lines:
        ax.axhline(hline, color='black', linestyle='--', linewidth=0.8)

    wafer_circle = plt.Circle((0, 0), wafer_radius, color='green', fill=False, linestyle='-', linewidth=2, label='Wafer Boundary')
    ax.add_patch(wafer_circle)

    scale_bar_length = 30000 * scale_factor
    scale_bar_label = f'{scale_bar_length * 1e3:.1f}nm'

    fontprops = fm.FontProperties(size=10)
    scalebar = AnchoredSizeBar(ax.transData, 30000, scale_bar_label, 'lower center', pad=0.1, color='black',
                               frameon=False, size_vertical=500, fontproperties=fontprops)
    ax.add_artist(scalebar)

    ax.set_xlabel('Wafer X Coordinate (wf_x)')
    ax.set_ylabel('Wafer Y Coordinate (wf_y)')
    ax.set_title(title)
    ax.axis('equal')
    ax.grid(False)

    return quiver

# 클릭한 플롯을 확대해서 보여주는 함수
class EnlargedPlotWindow(QMainWindow):
    def __init__(self, x, y, u, v, title, v_lines, h_lines, wafer_radius=150000, scale_factor=1e-7):
        super().__init__()
        self.setWindowTitle(title)
        self.setGeometry(100, 100, 800, 800)

        # 확대된 플롯을 새 창에 표시
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # 기존의 plot_overlay 함수 사용하여 가로선, 세로선, 원형 추가
        plot_overlay(ax, x, y, u, v, v_lines, h_lines, wafer_radius, title, scale_factor)

        ##### m3s 값을 계산하여 팝업 창에도 표시
        mean_plus_3sigma_x = calculate_statistics(u)
        mean_plus_3sigma_y = calculate_statistics(v)
        ax.text(0, -170000, f'|m|+3s X: {mean_plus_3sigma_x:.2f} nm\n|m|+3s Y: {mean_plus_3sigma_y:.2f} nm', 
                fontsize=10, color='red', ha='center')
        


        # Matplotlib FigureCanvas 생성
        self.canvas = FigureCanvas(fig)

        # Navigation Toolbar 추가
        self.toolbar = NavigationToolbar(self.canvas, self)

        layout = QVBoxLayout()
        layout.addWidget(self.toolbar)  # 툴바 추가
        layout.addWidget(self.canvas)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)


# 플롯을 보여주는 함수
class PlotWindow(QMainWindow):
    def __init__(self, unique_id, df):
        super().__init__()
        self.setWindowTitle(f"Plot for {unique_id}")
        self.setGeometry(100, 100, 1200, 800)

        # 데이터 필터링
        df_lot = df[df['UNIQUE_ID'] == unique_id]

        wf_x = df_lot['wf_x']
        wf_y = df_lot['wf_y']
        X_reg = df_lot['X_reg']
        Y_reg = df_lot['Y_reg']
        pred_x = df_lot['pred_x']
        pred_y = df_lot['pred_y']
        residual_x = df_lot['residual_x']
        residual_y = df_lot['residual_y']

        psm_fit_x = df_lot['psm_fit_x']
        psm_fit_y = df_lot['psm_fit_y']
        residual_x_depsm = df_lot['residual_x_depsm']
        residual_y_depsm = df_lot['residual_y_depsm']        
        cpe19p_pred_x = df_lot['cpe19p_pred_x']
        cpe19p_pred_y = df_lot['cpe19p_pred_y']
        cpe19p_resi_x = df_lot['cpe19p_resi_x']
        cpe19p_resi_y = df_lot['cpe19p_resi_y']
        ideal_psm_x = df_lot['ideal_psm_x']
        ideal_psm_y = df_lot['ideal_psm_y']

        delta_psm_x = df_lot['delta_psm_x']
        delta_psm_y = df_lot['delta_psm_y']

        step_pitch_x = df_lot['STEP_PITCH_X'].iloc[0]
        step_pitch_y = df_lot['STEP_PITCH_Y'].iloc[0]
        map_shift_x = df_lot['MAP_SHIFT_X'].iloc[0]
        map_shift_y = df_lot['MAP_SHIFT_Y'].iloc[0]
        start_left = -(step_pitch_x)/2 + map_shift_x
        start_bottom = -(step_pitch_y)/2 + map_shift_y
        max_die_x = max(df_lot['DieX'])
        min_die_x = min(df_lot['DieX'])
        max_die_y = max(df_lot['DieY'])
        min_die_y = min(df_lot['DieY'])

        self.vertical_lines = calculate_vlines(start_left, step_pitch_x, max_die_x, min_die_x)
        self.horizontal_lines = calculate_hlines(start_bottom, step_pitch_y, max_die_y, min_die_y)


        ### 확대된 창들을 저장할 리스트 ###
        self.enlarged_plot_windows = []

        # Figure 생성
        fig, axes = plt.subplots(4, 3, figsize=(12, 8))
        fig.suptitle(f'Visualizations for Lot {unique_id}', fontsize=16)

        self.quivers = []
        self.quivers.append((plot_overlay(axes[0, 0], wf_x, wf_y, X_reg, Y_reg, self.vertical_lines, self.horizontal_lines, title='Raw(X_reg,Y_reg)'), 'Raw(X_reg,Y_reg)'))
        self.quivers.append((plot_overlay(axes[0, 1], wf_x, wf_y, pred_x, pred_y, self.vertical_lines, self.horizontal_lines, title='OSR_Fitting(WK,RK)'), 'OSR_Fitting(WK,RK)'))
        self.quivers.append((plot_overlay(axes[0, 2], wf_x, wf_y, residual_x, residual_y, self.vertical_lines, self.horizontal_lines, title='Residual'), 'Residual'))
        self.quivers.append((plot_overlay(axes[1, 0], wf_x, wf_y, psm_fit_x, psm_fit_y, self.vertical_lines, self.horizontal_lines, title='PSM Input'), 'PSM Input'))
        self.quivers.append((plot_overlay(axes[1, 1], wf_x, wf_y, residual_x_depsm, residual_y_depsm, self.vertical_lines, self.horizontal_lines, title='Residual(Remove_PSM)'), 'Residual(Remove_PSM)'))
        self.quivers.append((plot_overlay(axes[1, 2], wf_x, wf_y, cpe19p_pred_x, cpe19p_pred_y, self.vertical_lines, self.horizontal_lines, title='CPE 19para Fitting'), 'CPE 19para Fitting'))
        self.quivers.append((plot_overlay(axes[2, 0], wf_x, wf_y, cpe19p_resi_x, cpe19p_resi_y, self.vertical_lines, self.horizontal_lines, title='CPE 19para Residual'), 'CPE 19para Residual'))
        self.quivers.append((plot_overlay(axes[2, 1], wf_x, wf_y, ideal_psm_x, ideal_psm_y, self.vertical_lines, self.horizontal_lines, title='Ideal PSM'), 'Ideal PSM'))
        self.quivers.append((plot_overlay(axes[2, 2], wf_x, wf_y, delta_psm_x, delta_psm_y, self.vertical_lines, self.horizontal_lines, title='Delta PSM'), 'Delta PSM'))
        self.quivers.append((plot_overlay(axes[3, 0], wf_x, wf_y, delta_psm_x, [0]*len(delta_psm_y), self.vertical_lines, self.horizontal_lines, title='Delta PSM X'), 'Delta PSM X'))
        self.quivers.append((plot_overlay(axes[3, 1], wf_x, wf_y, [0]*len(delta_psm_x), delta_psm_y, self.vertical_lines, self.horizontal_lines, title='Delta PSM Y'), 'Delta PSM Y'))





        # 통계치(ABS(Mean) + 3sigma) 계산 및 추가
        for i, (quiver, title) in enumerate(self.quivers):
            mean_plus_3sigma_x = calculate_statistics(quiver.U)
            mean_plus_3sigma_y = calculate_statistics(quiver.V)
            axes[i // 3, i % 3].text(0, -170000, f'|m|+3s X: {mean_plus_3sigma_x:.2f}nm\n|m|+3s Y: {mean_plus_3sigma_y:.2f}nm', 
                                     fontsize=10, color='red', ha='center')
            



 




        # FigureCanvas를 생성하고 레이아웃에 추가
        self.canvas = FigureCanvas(fig)

        # Navigation Toolbar 추가
        self.toolbar = NavigationToolbar(self.canvas, self)

        layout = QVBoxLayout()
        layout.addWidget(self.toolbar)  # 툴바 추가
        layout.addWidget(self.canvas)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # 클릭 이벤트 연결
        self.canvas.mpl_connect('button_press_event', self.on_click)


    def on_click(self, event):
        for i, (quiver, title) in enumerate(self.quivers):
            if event.inaxes and quiver.contains(event)[0]:  # 클릭한 객체가 quiver인지 확인
                # 클릭한 플롯을 확대하여 새 창으로 표시
                enlarged_window = EnlargedPlotWindow(
                    quiver.X, quiver.Y, quiver.U, quiver.V, 
                    title=title,  # 원래 플롯의 제목을 전달
                    v_lines=self.vertical_lines,  # 가로선
                    h_lines=self.horizontal_lines  # 세로선
                )
                # 새로 생성된 확대 창을 리스트에 저장
                self.enlarged_plot_windows.append(enlarged_window)
                enlarged_window.show()
                

         

# 메인 윈도우
class MainWindow(QMainWindow):
    def __init__(self, df):
        super().__init__()
        self.setWindowTitle("Unique ID Plot Viewer")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        # 라벨 추가
        label = QLabel("Select a unique_id to view the plot:")
        layout.addWidget(label)

        # 콤보박스 추가
        self.combo_box = QComboBox()
        self.combo_box.addItems(df['UNIQUE_ID'].unique())
        layout.addWidget(self.combo_box)

        # 버튼 추가
        button = QPushButton("Show Plot")
        button.clicked.connect(self.show_plot)
        layout.addWidget(button)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # 데이터 저장
        self.df = df

    def show_plot(self):
        unique_id = self.combo_box.currentText()
        self.plot_window = PlotWindow(unique_id, self.df)
        self.plot_window.show()


# PyQt 애플리케이션 실행
def main():
    file_path = 'output.xlsx'  # 파일 경로 수정 필요
    sheet_name = 'Delta_PSM'

    # Excel 데이터 로드
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    app = QApplication([])
    main_window = MainWindow(df)
    main_window.show()
    app.exec_()


if __name__ == "__main__":
    main()




